In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# from langgraph.checkpoint.memory import MemorySaver  # Ram me items ko store karti hai 
from langgraph.checkpoint.memory import InMemorySaver  # Ram me items ko store karti hai 
import os

In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!",
 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental health professional trained to help individuals cope with stress and pressure. The joke suggests that the pizza is seeking therapy to deal with the "pressure" of having too many toppings, which is a humorous and lighthearted way to address the idea of feeling overwhelmed. The absurdity of a pizza seeking therapy adds to the humor of the joke.'}

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental health professional trained to help individuals cope with stress and pressure. The joke suggests that the pizza is seeking therapy to deal with the "pressure" of having too many toppings, which is a humorous and lighthearted way to address the idea of feeling overwhelmed. The absurdity of a pizza seeking therapy adds to the humor of the joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1741-38a0-6f05-8002-a19f0674d79c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T18:08:49.480039+00:00', parent_config={'configurable': {'thread_

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental health professional trained to help individuals cope with stress and pressure. The joke suggests that the pizza is seeking therapy to deal with the "pressure" of having too many toppings, which is a humorous and lighthearted way to address the idea of feeling overwhelmed. The absurdity of a pizza seeking therapy adds to the humor of the joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1741-38a0-6f05-8002-a19f0674d79c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T18:08:49.480039+00:00', parent_config={'configurable': {'thread

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': "Why did the pasta go to the party alone? Because it couldn't find a date-nut sauce!",
 'explanation': 'This joke plays on the word "date" which can refer to both a social event or a sweet fruit. In this case, the pasta couldn\'t find a "date" (as in a romantic partner) to bring to the party, so it went alone. Additionally, "date-nut" sauce is a type of sauce that is commonly paired with certain dishes, so the joke implies that the pasta went to the party alone because it couldn\'t find the appropriate sauce to accompany it.'}

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental health professional trained to help individuals cope with stress and pressure. The joke suggests that the pizza is seeking therapy to deal with the "pressure" of having too many toppings, which is a humorous and lighthearted way to address the idea of feeling overwhelmed. The absurdity of a pizza seeking therapy adds to the humor of the joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1741-38a0-6f05-8002-a19f0674d79c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T18:08:49.480039+00:00', parent_config={'configurable': {'thread_

In [12]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental health professional trained to help individuals cope with stress and pressure. The joke suggests that the pizza is seeking therapy to deal with the "pressure" of having too many toppings, which is a humorous and lighthearted way to address the idea of feeling overwhelmed. The absurdity of a pizza seeking therapy adds to the humor of the joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1741-38a0-6f05-8002-a19f0674d79c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T18:08:49.480039+00:00', parent_config={'configurable': {'thread

# Time Travel

In [14]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})


StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [16]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})


In [17]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental health professional trained to help individuals cope with stress and pressure. The joke suggests that the pizza is seeking therapy to deal with the "pressure" of having too many toppings, which is a humorous and lighthearted way to address the idea of feeling overwhelmed. The absurdity of a pizza seeking therapy adds to the humor of the joke.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1741-38a0-6f05-8002-a19f0674d79c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T18:08:49.480039+00:00', parent_config={'configurable': {'thread

# Updating State

In [19]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})


{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0f1746-7ae6-6983-8000-657c4c98451e'}}

In [20]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1746-7ae6-6983-8000-657c4c98451e'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2026-01-14T18:11:10.646918+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='bc13176f-bc82-8e37-e17d-d15df2bc843f', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental h

In [22]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})


In [23]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1746-7ae6-6983-8000-657c4c98451e'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2026-01-14T18:11:10.646918+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='bc13176f-bc82-8e37-e17d-d15df2bc843f', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the psychiatrist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the concept of a pizza with "too many toppings" as a metaphor for feeling overwhelmed or stressed. In the joke, the pizza goes to see a psychiatrist, who is a mental h

# Fault Tolerance

In [25]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [26]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [28]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [29]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [30]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [32]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [33]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f174a-9a12-6d82-8001-f3c7d8211011'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-01-14T18:13:01.289806+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f174a-9a0d-6fd6-8000-0870a78bce05'}}, tasks=(PregelTask(id='7b0cdfe3-11ae-b887-ff26-79b2b438f794', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f174a-9a0d-6fd6-8000-0870a78bce05'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2026-01-14T18:13:01.287812+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'ch